<a href="https://colab.research.google.com/github/sandeep240288/streamlit/blob/main/ChartInkFilterWeeklyScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installations
!pip install bs4
!pip install yfinance
!pip install pandas_ta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=57530f3a4104eae68bd48305b9dcba1f054faf27dfad7559034289319908026a
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=412720ff74a5e3b208270d4ef30dec3e4434a933a5560f96525458756d5a8a85
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0


In [2]:
# Import
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import yfinance as yf

In [5]:
# filtered Symbols
url = "https://chartink.com/screener/process"

condition_friday = {"scan_clause": "( {cash} ( weekly close >= 30 and latest sma( latest close , 10 ) >= latest sma( latest close , 30 ) and latest sma( latest volume , 5 ) >= 10000 and market cap > 0 and weekly sma( weekly close , 40 ) <= latest close and latest \"\close - 1 candle ago close / 1 candle ago close * 100\"\ >= -4 ) )"}
condition_consolidating = {"scan_clause": "( {cash} ( weekly close >= 30 and latest sma( latest close , 10 ) >= latest sma( latest close , 30 ) and latest sma( latest volume , 5 ) >= 10000 and market cap > 0 and weekly sma( weekly close , 40 ) <= latest close and latest \"\close - 1 candle ago close / 1 candle ago close * 100\"\ >= -4 and( {cash} ( ( abs( latest close - latest sma( latest close , 9 ) ) / latest sma( latest close , 9 ) ) <= .02 or( abs( latest close - latest sma( latest close , 20 ) ) / latest sma( latest close , 20 ) ) <= .02 ) ) ) ) "}

condition = condition_friday

with requests.session() as s:
    r_data = s.get(url)
    soup = bs(r_data.content, "lxml")
    meta = soup.find("meta", {"name" : "csrf-token"})["content"]

    header = {"x-csrf-token" : meta}
    data = s.post(url, headers=header, data=condition).json()

    query_stock_list = pd.DataFrame(data["data"])


In [ ]:
# Yfinance Symbols
stock_nse_codes = query_stock_list['nsecode'].to_list()
stocks_nse_symbol = [symbol+'.NS' for symbol in stock_nse_codes]
stocks_nse_symbol

In [7]:
# date range to fetch Data from
from datetime import date ,timedelta
today = date.today()+timedelta(days=1)
start_date = today-timedelta(days=180)
start_date

datetime.date(2022, 12, 18)

In [8]:
stock_list = stocks_nse_symbol
filtered_stocks_purple_dots =[]
filtered_darvax =[]

In [9]:
# function definitions for analysis
def darvax(df,n,range_pct):
  rolling_max  = df['Close'].rolling(window=n).max()
  rolling_min = df['Close'].rolling(window=n).min()

  last_max = rolling_max[-1]
  last_min = rolling_min[-1]

  rang_pct = 100*(last_max-last_min)/last_min
  return rang_pct < range_pct



def stock_range(close,open):
    return close-open



def nr3(df):
  close1 = df['Close'][-1]
  open1= df['Open'][-1]

  close2 = df['Close'][-2]
  open2 = df['Open'][-2]

  close3 = df['Close'][-3]
  open3 = df['Open'][-3]


  r1 = stock_range(close1,open1)
  r2 = stock_range(close2,open2)
  r3 = stock_range(close3,open3)

  if close3>close2>close1 and open3<open2<open1:
    return True
  elif r3>r2>r1:
     return True
  return False


In [10]:
import multiprocessing as mp
def get_stock_history(symbol):
  """Gets the stock history for the given symbol."""
  ticker = yf.Ticker(symbol)
  if ticker:
    stock_history = ticker.history(start=start_date, end=today, interval="1d")
    stock_history["return"] = stock_history["Close"].pct_change() * 100
    stock_history["is_1M"] = stock_history["Volume"] >= 1000000
    stock_history["gt_5pct"] = stock_history["return"] >= 5
    stock_history["match"] = stock_history["is_1M"] & stock_history["gt_5pct"]
    total = stock_history["match"].sum()
    stock_history["total"] = total
    stock_history["sma10"] = stock_history["Close"].rolling(10).mean()
    stock_history["sma10_distance"] =abs((stock_history["sma10"]/stock_history["Close"])-1)*100
    zerodha_symbol = symbol.split(".", 1)[0]
    return stock_history, total, zerodha_symbol
  else:
    return None, None, None

In [11]:

def main(stock_list):
  """The main function."""
  # Create a pool of workers.
  pool = mp.Pool()

  # Get the stock histories for all of the symbols.
  results = pool.map(get_stock_history, stock_list)

  # Filter the results.
  filtered_stocks_purple_dots = []
  stocks_at_10_ma = []
  for result in results:
    if result is not None and  result[1] >= 2:
        if result[0].iloc[-1]['sma10_distance']<=1:
          stocks_at_10_ma.append(result[2])

        filtered_stocks_purple_dots.append({
            "symbol": result[2],
            "total": result[1]
        })



  # Print the filtered results.
  return filtered_stocks_purple_dots,stocks_at_10_ma


In [12]:
filtered_stocks_purple_dots,stocks_at_10_ma = main(stock_list)

Failed to create TzCache, reason: Error creating TzCache folder: '/root/.cache/py-yfinance' reason: [Errno 17] File exists: '/root/.cache/py-yfinance'
TzCache will not be used.
Tip: You can direct cache to use a different location with 'set_tz_cache_location(mylocation)'


In [13]:
stocks_at_10_ma

['PERSISTENT',
 'HDFCAMC',
 'NUCLEUS',
 'BDL',
 'HAPPSTMNDS',
 'GLENMARK',
 'GRAVITA',
 'KIRLOSBROS',
 'HDFCLIFE',
 'DATAMATICS',
 'BORORENEW',
 'HINDWAREAP',
 'ACE',
 'BCLIND',
 'MAHSEAMLES',
 'CHEMPLASTS',
 'AARTIDRUGS',
 'IOLCP',
 'ZENTEC',
 'RATEGAIN',
 'ZENSARTECH',
 'PRAJIND',
 'BFINVEST',
 'CGPOWER',
 'BFUTILITIE',
 'BSOFT',
 'KOLTEPATIL',
 'FCL',
 'BANCOINDIA',
 'GSPL',
 'M&MFIN',
 'TRIVENI',
 'SCHNEIDER',
 'UGROCAP',
 'EIHOTEL',
 'EPL',
 'BANKBARODA',
 'GICRE',
 'BHARATWIRE',
 'JAICORPLTD',
 'ANANTRAJ',
 'STARCEMENT',
 'ORIENTCEM',
 'FSL',
 'ARVIND',
 'PRSMJOHNSN',
 'DCAL',
 'NCC',
 'KOKUYOCMLN',
 'ENGINERSIN',
 'KECL',
 'ITI',
 'NLCINDIA',
 'MAWANASUG',
 'NACLIND',
 'MARKSANS',
 'NATIONALUM',
 'SAIL',
 'BHEL',
 'IRCON',
 'TRIL',
 'PENIND',
 'ZOMATO',
 'UNIONBANK',
 'GOLDTECH',
 'MARSHALL',
 'PNB',
 'MARINE',
 'TPLPLASTEH',
 'TV18BRDCST',
 'TRIDENT']

In [14]:
# symbol1 ='DIXON.NS'
# ticker = yf.Ticker(symbol1)
# if ticker:
#   stock_history = ticker.history(start=start_date, end=today, interval="1d")

In [15]:
# stock_history.iloc[-1]

In [16]:
# stock_history.tail()

In [17]:
for symbol in list(stocks_at_10_ma):
  print(symbol+',')

PERSISTENT,
HDFCAMC,
NUCLEUS,
BDL,
HAPPSTMNDS,
GLENMARK,
GRAVITA,
KIRLOSBROS,
HDFCLIFE,
DATAMATICS,
BORORENEW,
HINDWAREAP,
ACE,
BCLIND,
MAHSEAMLES,
CHEMPLASTS,
AARTIDRUGS,
IOLCP,
ZENTEC,
RATEGAIN,
ZENSARTECH,
PRAJIND,
BFINVEST,
CGPOWER,
BFUTILITIE,
BSOFT,
KOLTEPATIL,
FCL,
BANCOINDIA,
GSPL,
M&MFIN,
TRIVENI,
SCHNEIDER,
UGROCAP,
EIHOTEL,
EPL,
BANKBARODA,
GICRE,
BHARATWIRE,
JAICORPLTD,
ANANTRAJ,
STARCEMENT,
ORIENTCEM,
FSL,
ARVIND,
PRSMJOHNSN,
DCAL,
NCC,
KOKUYOCMLN,
ENGINERSIN,
KECL,
ITI,
NLCINDIA,
MAWANASUG,
NACLIND,
MARKSANS,
NATIONALUM,
SAIL,
BHEL,
IRCON,
TRIL,
PENIND,
ZOMATO,
UNIONBANK,
GOLDTECH,
MARSHALL,
PNB,
MARINE,
TPLPLASTEH,
TV18BRDCST,
TRIDENT,


In [18]:
# calculate logic for reversal play here

In [19]:
!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from tabulate import tabulate
import operator
def print_table(dict):
    print(tabulate(dict, headers="keys", tablefmt="fancy_grid"))

In [21]:
dict = sorted(filtered_stocks_purple_dots, key=operator.itemgetter('total'), reverse=True)

# dict = sorted(filtered_stocks_purple_dots.items(), key=lambda x: x[1])

# print_table(dict)
n = 4 #Greater than n
filtered_list = list(filter(lambda dict: dict['total'] > n, dict))

print_table(filtered_list)


# print(filtered_stocks_purple_dots)

╒════════════╤═════════╕
│ symbol     │   total │
╞════════════╪═════════╡
│ JINDALSAW  │      16 │
├────────────┼─────────┤
│ JBMA       │      12 │
├────────────┼─────────┤
│ ZENTEC     │      12 │
├────────────┼─────────┤
│ EKC        │      12 │
├────────────┼─────────┤
│ FACT       │      11 │
├────────────┼─────────┤
│ JISLJALEQS │      11 │
├────────────┼─────────┤
│ USHAMART   │      10 │
├────────────┼─────────┤
│ BHARATWIRE │      10 │
├────────────┼─────────┤
│ TRIL       │      10 │
├────────────┼─────────┤
│ VASCONEQ   │      10 │
├────────────┼─────────┤
│ HBLPOWER   │       9 │
├────────────┼─────────┤
│ PENIND     │       9 │
├────────────┼─────────┤
│ RADHIKAJWE │       9 │
├────────────┼─────────┤
│ KPITTECH   │       8 │
├────────────┼─────────┤
│ LODHA      │       8 │
├────────────┼─────────┤
│ DATAMATICS │       8 │
├────────────┼─────────┤
│ ELGIEQUIP  │       8 │
├────────────┼─────────┤
│ MAHINDCIE  │       8 │
├────────────┼─────────┤
│ TWL        │       8 │


In [22]:
# filtered_stocks_purple_dots

Backup Code for getting data from yfinance
```
# This is formatted as code
  df_data ={}
  purple_dots_data = {}
  for symbol in stock_list:
    ticker = yf.Ticker(symbol)

    if ticker:
      stock_history = ticker.history(start=start_date,end=today, interval="1d")
      stock_history["return"] = stock_history["Close"].pct_change()*100
      stock_history["is_1M"] = stock_history["Volume"]>=1000000
      stock_history["gt_5pct"] = stock_history["return"]>=5
      stock_history["match"] = stock_history["is_1M"] & stock_history["gt_5pct"]
      total = stock_history["match"].sum()
      stock_history["total"] = total
      zerodha_symbol = symbol.split('.',1)[0]
      df_data[zerodha_symbol] = stock_history
      if total>=2:
        filtered_stocks_purple_dots.append({'symbol':zerodha_symbol,'total':total})
        df_data[zerodha_symbol] = stock_history
      if darvax(stock_history,14,5):
        filtered_darvax.append(zerodha_symbol)
```

In [23]:
filtered_stocks_purple_dots
filtered_list

[{'symbol': 'JINDALSAW', 'total': 16},
 {'symbol': 'JBMA', 'total': 12},
 {'symbol': 'ZENTEC', 'total': 12},
 {'symbol': 'EKC', 'total': 12},
 {'symbol': 'FACT', 'total': 11},
 {'symbol': 'JISLJALEQS', 'total': 11},
 {'symbol': 'USHAMART', 'total': 10},
 {'symbol': 'BHARATWIRE', 'total': 10},
 {'symbol': 'TRIL', 'total': 10},
 {'symbol': 'VASCONEQ', 'total': 10},
 {'symbol': 'HBLPOWER', 'total': 9},
 {'symbol': 'PENIND', 'total': 9},
 {'symbol': 'RADHIKAJWE', 'total': 9},
 {'symbol': 'KPITTECH', 'total': 8},
 {'symbol': 'LODHA', 'total': 8},
 {'symbol': 'DATAMATICS', 'total': 8},
 {'symbol': 'ELGIEQUIP', 'total': 8},
 {'symbol': 'MAHINDCIE', 'total': 8},
 {'symbol': 'TWL', 'total': 8},
 {'symbol': 'TRITURBINE', 'total': 8},
 {'symbol': 'MOLDTECH', 'total': 8},
 {'symbol': 'NAVA', 'total': 8},
 {'symbol': 'ICIL', 'total': 8},
 {'symbol': 'RVNL', 'total': 8},
 {'symbol': 'SCI', 'total': 8},
 {'symbol': 'BOMDYEING', 'total': 8},
 {'symbol': 'RICOAUTO', 'total': 8},
 {'symbol': 'IRCON', 't

In [24]:
purple_dots_full = [symbol['symbol'] for symbol in filtered_stocks_purple_dots]
purple_dots_filtered = [symbol['symbol'] for symbol in filtered_list]

In [25]:
for symbol in list(purple_dots_filtered):
  print(symbol+',')

JINDALSAW,
JBMA,
ZENTEC,
EKC,
FACT,
JISLJALEQS,
USHAMART,
BHARATWIRE,
TRIL,
VASCONEQ,
HBLPOWER,
PENIND,
RADHIKAJWE,
KPITTECH,
LODHA,
DATAMATICS,
ELGIEQUIP,
MAHINDCIE,
TWL,
TRITURBINE,
MOLDTECH,
NAVA,
ICIL,
RVNL,
SCI,
BOMDYEING,
RICOAUTO,
IRCON,
ISMTLTD,
TEXRAIL,
CYIENT,
PAYTM,
PNBHOUSING,
TVSELECT,
SAKSOFT,
MINDACORP,
SDBL,
IZMO,
HINDOILEXP,
CAPACITE,
PRECAM,
GICRE,
SURYODAY,
ITDCEM,
ARVIND,
DCAL,
EQUITASBNK,
HUDCO,
IMAGICAA,
SALASAR,
PSB,
MAZDOCK,
SONATSOFTW,
OLECTRA,
APTECHT,
ZENSARTECH,
CHENNPETRO,
JSL,
SCHNEIDER,
UGROCAP,
TDPOWERSYS,
BLS,
NRBBEARING,
JAICORPLTD,
KALYANKJIL,
HIMATSEIDE,
ENGINERSIN,
KECL,
WELSPUNIND,
MADRASFERT,
CONFIPET,
RAMASTEEL,
TRIDENT,
KEC,
COCHINSHIP,
BORORENEW,
REFEX,
ACE,
RKFORGE,
UJJIVAN,
EIHOTEL,
FOODSIN,
MANAKSIA,
ANANTRAJ,
MANINDS,
HSCL,
RCF,
GENUSPOWER,
MAWANASUG,
MARKSANS,
WALCHANNAG,
NETWORK18,
IDBI,
MARINE,
MANAKSTEEL,
TV18BRDCST,


In [26]:
filtered_darvax

[]

In [ ]:
# a_set = set(filtered_darvax)
# b_set = set(purple_dots)


# common_sets = a_set&b_set

# list(common_sets)
# for symbol in list(common_sets):
#   print(symbol+',')